### Adjust Word Group In Youtube Sentence Word Usage Analysis2

This notebook is used to limit the sentence and sentence groups obtained in the previous notebook (2-Word Group In Youtube Rev2.ipynb) according to word usage. (Analysis2)

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# word all usage in sent
word_use_num_max = 3
word_use_num_min = 1

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/4-Adjust Word Group In Youtube Sentence Word Usage Analysis2/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [8]:
df_youtube_sent_manual_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/\
3-Adjust Word Group In Youtube Sentence Word Usage Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_youtube_sent_manual_select.sort_values(by="search_string", key=lambda x:x.str.len(), ascending=True, inplace=True)
df_youtube_sent_manual_select.drop(["word_index"], axis=1, inplace=True)
#df_youtube_sent_result = df_youtube_sent_result.sample(107)
df_youtube_sent_manual_select

,search_string,start_time,end_time,sentence,video_id,video_url
70,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...
69,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...
68,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...
67,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...
66,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s
...,...,...,...,...,...,...
4,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...
3,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...


In [9]:
df_youtube_sent_manual_select["word_index_list"] = df_youtube_sent_manual_select["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_manual_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
70,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
69,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
68,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
67,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...,"[112, 71, 44, 172]"
66,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
...,...,...,...,...,...,...,...
4,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
3,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."


In [10]:
df_youtube_sent_select = df_youtube_sent_manual_select[df_youtube_sent_manual_select["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
1,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
2,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
3,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...,"[112, 71, 44, 172]"
4,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
...,...,...,...,...,...,...,...
102,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
103,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
104,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
105,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."


In [11]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [12]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [13]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[131,
 137,
 12,
 140,
 15,
 24,
 154,
 155,
 157,
 159,
 163,
 36,
 165,
 169,
 171,
 173,
 50,
 178,
 182,
 183,
 185,
 59,
 65,
 67,
 199,
 72,
 80,
 82,
 85,
 101,
 105,
 106,
 115,
 117,
 124]

In [14]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index_list"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
1,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
2,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
3,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...,"[112, 71, 44, 172]"
4,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
5,diye söyle bak hala,104,106,o taşlarını gecesi zor için yoğurduktan sonra ...,P5bYtFrqw68,https://www.youtube.com/watch?v=P5bYtFrqw68&t=...,"[123, 161, 64, 167]"
6,olan oldu artık geri,110,112,olan oldu artık geri kalanı,3F9rGMJaC2Y,https://www.youtube.com/watch?v=3F9rGMJaC2Y&t=...,"[79, 55, 61, 63]"
7,içinde tam işte uzun,303,305,çıkıyor içinde tam işte uzun çıkmıyor para çık...,n5b8T70avq8,https://www.youtube.com/watch?v=n5b8T70avq8&t=...,"[158, 112, 108, 172]"
8,en son şu dakika önce,646,649,en son şu dakika önce alındı kampanya bitmek ü...,15Ra_1BpFDM,https://www.youtube.com/watch?v=15Ra_1BpFDM&t=...,"[44, 102, 48, 177, 51]"
9,biri büyük biri küçük,150,152,biri büyük biri küçük olduğu için siz elmanını...,2Zay9LyI1aQ,https://www.youtube.com/watch?v=2Zay9LyI1aQ&t=...,"[78, 76, 78, 125]"


In [15]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list 

In [16]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['ver',
 'haydi',
 'evet',
 'izin',
 'da',
 'seni',
 'vardı',
 'değilim',
 'benimle',
 'baba',
 'mü',
 'senin',
 'selam',
 'al',
 'üç',
 'herkes',
 'hadi',
 'para',
 'bence',
 'gerçek',
 'pekala',
 'bile',
 'istiyorum',
 'eğer',
 'ister',
 'tek',
 'olur',
 'hey',
 'efendim',
 'yeni',
 'kötü',
 'gece',
 'harika',
 'gerek',
 'gel']

In [17]:
len(not_in_sent_word)

35

In [18]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for df_target all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [19]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_word, df_youtube_sent_select, "search_string", 1, simple=False)
not_in_sent_selected_word_list_result = not_in_sent_selected_word_list_result.drop(["word"], axis=1)
not_in_sent_selected_word_list_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kadar bana biraz zaman ver ben biraz,610,614,de istiyorum ve ben yaz aylarına kadar bana bi...,cXMr5QBndeA,https://www.youtube.com/watch?v=cXMr5QBndeA&t=...,"[20, 21, 52, 35, 131, 7, 52]"
1,haydi bakalım şimdi biz bunu çünkü,590,594,haydi bakalım şimdi biz bunu çünkü ayırdık ama...,RISpYv3UB3k,https://www.youtube.com/watch?v=RISpYv3UB3k&t=...,"[137, 175, 43, 88, 26, 68]"
2,evet tamam çok güzel hadi bakalım şimdi şey,272,280,evet tamam çok güzel hadi bakalım şimdi şey ya...,s5Qt_BegRJo,https://www.youtube.com/watch?v=s5Qt_BegRJo&t=...,"[12, 29, 9, 54, 50, 175, 43, 16]"
3,peki yani siz en çok böyle izin,282,284,peki yani siz en çok böyle izin almadan hareke...,ZSX8Onpr7Qw,https://www.youtube.com/watch?v=ZSX8Onpr7Qw&t=...,"[66, 60, 119, 44, 9, 56, 140]"
4,biri küçük o da kötü bir şey olur ya şey seni bir,35,38,yere içeri parmakları serçe parmağından biri k...,myC7xKM1Btk,https://www.youtube.com/watch?v=myC7xKM1Btk&t=35s,"[78, 125, 6, 15, 105, 0, 16, 80, 45, 16, 24, 0]"
5,kız seni gel buraya gel buraya kız gel,125,129,saymadın beni çok özledim kız seni gel buraya ...,Y5kxIA77e1E,https://www.youtube.com/watch?v=Y5kxIA77e1E&t=...,"[138, 24, 124, 77, 124, 77, 138, 124]"
6,var değil mi mi ben çok çok yok vardı var bir de,84,87,ülkelerde yoğunluk var değil mi mi ben çok çok...,cUfp2lydmho,https://www.youtube.com/watch?v=cUfp2lydmho&t=84s,"[11, 14, 5, 5, 7, 9, 9, 22, 154, 11, 0, 8]"
7,musun bilmiyorum değilim neden peki yani,377,381,arada mutlu musun bilmiyorum değilim neden pek...,paJ2IFbflzU,https://www.youtube.com/watch?v=paJ2IFbflzU&t=...,"[53, 94, 155, 34, 66, 60]"
8,söyle bakalım benimle şimdi geliyor musun,207,211,ee söyle bakalım benimle şimdi geliyor musun,9RP0Faoo2rw,https://www.youtube.com/watch?v=9RP0Faoo2rw&t=...,"[161, 175, 157, 43, 179, 53]"
9,bir daha baba tamam lütfen ama ya tamam,678,682,bir daha baba tamam lütfen ama ya tamam,GQgS2ruEJAs,https://www.youtube.com/watch?v=GQgS2ruEJAs&t=...,"[0, 18, 159, 29, 58, 10, 45, 29]"


In [20]:
df_result_concat = pd.concat([df_youtube_sent_sample, not_in_sent_selected_word_list_result], axis=0)
df_result_concat.sort_values(by="search_string", key=lambda x:x.str.len(), ascending=False, inplace=True)
df_result_concat.reset_index(drop=True, inplace=True)
df_result_concat

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
3,çok iyi değil mi tam böyle gerçek gibi gerçek ...,3,6,çok iyi değil mi tam böyle gerçek gibi gerçek ...,OmT_BtZh8qU,https://www.youtube.com/watch?v=OmT_BtZh8qU&t=3s,"[9, 28, 14, 5, 112, 56, 183, 27, 183, 27, 60]"
4,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[39, 0, 181, 40, 3, 34, 6, 181, 40]"
...,...,...,...,...,...,...,...
84,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
85,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...,"[112, 71, 44, 172]"
86,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
87,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"


In [21]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [22]:
df_shared_word_manuel_count = word_count_result(df_result_concat, df_result_concat.columns[0:1])
df_shared_word_manuel_count.sort_values(by="word_count", inplace=True, ascending=False)
df_shared_word_manuel_count

,word,word_count
0,bir,17
1,çok,15
2,şey,11
3,bakalım,10
4,güzel,9
...,...,...
161,doğru,1
162,dostum,1
163,bütün,1
164,sanırım,1


In [23]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [24]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'gece', 'pekala'}

In [25]:
df_result_concat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample3.xlsx", index=False)

In [26]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample3_Word_Count.xlsx", index=False)

#### Copy Move And Delete

In [27]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Turkish_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample3.xlsx',
 'Turkish_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample3_Word_Count.xlsx']

In [28]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [29]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass